## Analyze CSV file

In [ ]:
import os

os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages "
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.6,"
    "org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.6 "
    "pyspark-shell"
)

from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("read_test_stream") \
        .getOrCreate()

final_df = spark.read.csv("/home/guest/notebooks/data/output/date=*/hour=*", header=True, inferSchema=True)
final_df.take(5)

from pyspark.sql.functions import col

final_df.orderBy(col("window_start").asc()).coalesce(1).write \
    .option("header", "true") \
    .mode("overwrite") \
    .csv("/home/guest/notebooks/final_output/")

:: loading settings :: url = jar:file:/home/guest/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/guest/.ivy2/cache
The jars for the packages stored in: /home/guest/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-488b7ec5-ef72-42fc-ae3c-30aa7ba17c7a;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.6 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.6 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in c

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob

In [ ]:
# Using '*' pattern 
print('\nFile name:\n')
for name in glob.glob('/home/guest/notebooks/final_output/*.csv'):
    print(name)

In [ ]:
pandas_df = pd.read_csv(f"{name}")
pandas_df.tail()

In [ ]:
pandas_df["datetime"] = pd.to_datetime(pandas_df["window_end"])

In [ ]:
sorted_df = pandas_df.sort_values(by='datetime').set_index("datetime")

In [ ]:
sorted_df['range_max_min'] = sorted_df['max'] - sorted_df['min']

In [ ]:
sorted_df.groupby("sensor").count()

In [ ]:
df_avg = sorted_df[["sensor", "avg"]].pivot(columns='sensor', values='avg')

#sorted_df.groupby("sensor").avg_obs.plot(figsize=(15,5))
#plt.legend()
#plt.plot()

plt.figure(figsize=(15, 2))
sns.heatmap(df_avg.T, 
            annot=True, 
            cbar=False, 
            linewidths=1, 
            linecolor='white', )
plt.title("AVG Value Per Minute")
plt.show()

In [ ]:
df_range = sorted_df[["sensor", "range_max_min"]].pivot(columns='sensor', values='range_max_min')

#sorted_df.groupby("sensor").range_max_min.plot(figsize=(15,5))
#plt.legend()
#plt.plot()

plt.figure(figsize=(15, 2))
sns.heatmap(df_range.T, 
            annot=True, 
            cbar=False, 
            linewidths=1, 
            linecolor='white')
plt.title("RANGE Per Minute")
plt.show()

In [ ]:
df_counts = sorted_df[["sensor", "count"]].pivot(columns='sensor', values='count')

plt.figure(figsize=(15, 2))
sns.heatmap(df_counts.T, annot=True, cbar=False, 
                    linewidths=1, 
                    linecolor='white', )
plt.title("OBS COUNT Per Minute")
plt.show()

In [ ]:
df_misses = sorted_df[["sensor", "count_miss"]].pivot(columns='sensor', values='count_miss')

plt.figure(figsize=(15, 2))
sns.heatmap(df_misses.T, annot=True, cbar=False, 
                    linewidths=1, 
                    linecolor='white', )
plt.title("COUNT MISSING")
plt.show()


In [ ]:
df_out_control = sorted_df[["sensor", "is_out_control"]].pivot(columns='sensor', values='is_out_control').fillna(0).astype(int)

plt.figure(figsize=(15, 2))
sns.heatmap(df_out_control.T, 
            annot=True, 
            cbar=True, 
            linewidths=1, 
            linecolor='white')
plt.title("OUT-OF-CONTROL: NORMAL-0, WARNING-1, ALARM-2")
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
sorted_df.groupby("sensor").max_delay.hist(legend=True)
plt.title("DELAY in observations")
plt.show()